In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

import pandas

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):

        super(NN, self).__init__()

        self.fc1 = nn.Linear(input_size, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.bn2 = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(p=0.2)
        self.activation = nn.LeakyReLU(negative_slope=0.01)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 128
num_epochs = 50

In [4]:
transform1 = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
])

transform2 = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transform1, download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transform2, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [5]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0)

In [9]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device=device)
        targets = targets.to(device=device)

        data = data.reshape(data.shape[0], -1)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

    scheduler.step()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:05<00:00, 92.83it/s]


In [10]:
@torch.inference_mode()
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    for x, y in loader:
        x = x.to(device=device)
        y = y.to(device=device)

        x = x.reshape(x.shape[0], -1)
        
        scores = model(x)
        _, predictions = scores.max(1)

        num_correct += (predictions == y).sum()
        num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [11]:
check_accuracy(train_loader, model)

tensor(0.9918, device='cuda:0')

In [12]:
check_accuracy(test_loader, model)

tensor(0.9942, device='cuda:0')

In [13]:
@torch.inference_mode()
def generate_predictions_csv(loader, model, device, output_csv):
    model.eval()
    predictions_list = []

    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        y = y.to(device=device)
        
        x = x.reshape(x.shape[0], -1)

        scores = model(x)
        _, predictions = scores.max(1)

        for i in range(len(predictions)):
            predictions_list.append([idx * len(x) + i, predictions[i].item()])

    predictions_df = pandas.DataFrame(predictions_list, columns=['ID', 'target'])
    predictions_df.to_csv(output_csv, index=False)

    model.train()

In [14]:
generate_predictions_csv(test_loader, model, device, output_csv='results.csv')